Deteksi sudut pada gerakan dasar taekwondo menggunakan Pose Estimation MediaPipe

Instalasi Library

In [2]:
%pip install mediapipe opencv-python



[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import modul yang dibutuhakan dari Library

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
from datetime import datetime
from tkinter import filedialog, Tk

mp_drawing = mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

Rumus hitung sudut (Arctan/Atan)

In [2]:
## Rumus hitung sudut (arctan)
def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    
    radians=np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    
    angle = round(angle)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

Program Image Target

In [3]:
# Load the image

# File Chooser
# Buat root jendela Tkinter
root = Tk()
root.withdraw()  # Sembunyikan jendela root

# Munculkan Dialog File Chooser
root.attributes('-topmost', True)
image_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])

if image_path == "" or None:
    print("Anda tidak memilih gamabar")
else:
    root.attributes('-topmost', False)

    image = cv2.imread(image_path)
    # Kalkulasi skala dimensi x dan y
    scale_x = image.shape[1]  # Lebar/ Width
    scale_y = image.shape[0]  # Tinggi/ Height

    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            
        # Recolor image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # simpan sudut - sudut target kedalam array
        angles_target = []

        # Extract landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Get landmarks coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Hitung sudut (vector)
            angle_left_elbow = calculate_angle(left_shoulder, left_elbow, left_wrist)
            angles_target.append(angle_left_elbow) # Siku Kiri

            angle_right_elbow = calculate_angle(right_shoulder, right_elbow, right_wrist)
            angles_target.append(angle_right_elbow) # Siku Kanan

            angle_left_shoulder = calculate_angle(left_elbow, left_shoulder, left_hip)
            angles_target.append(angle_left_shoulder) # Bahu Kiri

            angle_right_shoulder = calculate_angle(right_elbow, right_shoulder, right_hip)
            angles_target.append(angle_right_shoulder) # Bahu Kanan

            angle_left_hip = calculate_angle(left_shoulder, left_hip, left_knee)
            angles_target.append(angle_left_hip) # Pinggul Kiri

            angle_right_hip = calculate_angle(right_shoulder, right_hip, right_knee)
            angles_target.append(angle_right_hip) # Pinggul Kanan

            angle_left_knee = calculate_angle(left_hip, left_knee, left_ankle)
            angles_target.append(angle_left_knee) # Lutut Kiri

            angle_right_knee = calculate_angle(right_hip, right_knee, right_ankle)
            angles_target.append(angle_right_knee) # Lutut Kanan

            # Visualize left elbow angle
            cv2.putText(image, str(angle_left_elbow), 
                            tuple(np.multiply(left_elbow, [scale_x, scale_y]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                        
            # Visualize right elbow angle
            cv2.putText(image, str(angle_right_elbow), 
                            tuple(np.multiply(right_elbow, [scale_x, scale_y]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Visualize left shoulder angle
            cv2.putText(image, str(angle_left_shoulder), 
                            tuple(np.multiply(left_shoulder, [scale_x, scale_y]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                        
            # Visualize right shoulder angle
            cv2.putText(image, str(angle_right_shoulder), 
                            tuple(np.multiply(right_shoulder, [scale_x, scale_y]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

            # Visualize left hip angle
            cv2.putText(image, str(angle_left_hip), 
                            tuple(np.multiply(left_hip, [scale_x, scale_y]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Visualize right hip angle
            cv2.putText(image, str(angle_right_hip), 
                            tuple(np.multiply(right_hip, [scale_x, scale_y]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Visualize left knee angle
            cv2.putText(image, str(angle_left_knee), 
                            tuple(np.multiply(left_knee, [scale_x, scale_y]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Visualize right knee angle
            cv2.putText(image, str(angle_right_knee), 
                            tuple(np.multiply(right_knee, [scale_x, scale_y]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
        else:
            print("Pose tidak terdeteksi")

        # print(angles_target)
        angle_names = {
            'Sudut Siku Kiri': 0,
            'Sudut Siku Kanan': 1,
            'Sudut Bahu Kiri': 2,
            'Sudut Bahu Kanan': 3,
            'Sudut Pinggul Kiri': 4,
            'Sudut Pinggul Kanan': 5,
            'Sudut Lutut Kiri': 6,
            'Sudut Lutut Kanan': 7
        }

        for angle_name, angle_index in angle_names.items():
            print(f"{angle_name}: {angles_target[angle_index]}")

        print("Array sudut target : ", angles_target)

        current_datetime = datetime.now() # Ambil tanggal dan waktu sekarang
        timestamp = current_datetime.strftime('%Y-%m-%d_%H-%M-%S') # Atur format tanggal dan ubah ke string

        output_image_path = f'Asset/Setelah Diproses/Gambar Target/Gambar_Processed_{timestamp}.jpg' #Ubah direktori untuk menyimpan gambar
        cv2.imwrite(output_image_path, image)  
        print(f"Processed image saved as '{output_image_path}'")
        
        cv2.imshow('Mediapipe Image Target', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

Sudut Siku Kiri: 91
Sudut Siku Kanan: 99
Sudut Bahu Kiri: 111
Sudut Bahu Kanan: 126
Sudut Pinggul Kiri: 178
Sudut Pinggul Kanan: 176
Sudut Lutut Kiri: 179
Sudut Lutut Kanan: 180
Array sudut target :  [91, 99, 111, 126, 178, 176, 179, 180]
Processed image saved as 'Asset/Setelah Diproses/Gambar Target/Gambar_Processed_2024-02-15_20-50-04.jpg'


Test Visualisasi

In [4]:
for idx, lnmark in enumerate(mp_pose.PoseLandmark):
    print(f"Landmark {idx + 1}: {lnmark.name}")

Landmark 1: NOSE
Landmark 2: LEFT_EYE_INNER
Landmark 3: LEFT_EYE
Landmark 4: LEFT_EYE_OUTER
Landmark 5: RIGHT_EYE_INNER
Landmark 6: RIGHT_EYE
Landmark 7: RIGHT_EYE_OUTER
Landmark 8: LEFT_EAR
Landmark 9: RIGHT_EAR
Landmark 10: MOUTH_LEFT
Landmark 11: MOUTH_RIGHT
Landmark 12: LEFT_SHOULDER
Landmark 13: RIGHT_SHOULDER
Landmark 14: LEFT_ELBOW
Landmark 15: RIGHT_ELBOW
Landmark 16: LEFT_WRIST
Landmark 17: RIGHT_WRIST
Landmark 18: LEFT_PINKY
Landmark 19: RIGHT_PINKY
Landmark 20: LEFT_INDEX
Landmark 21: RIGHT_INDEX
Landmark 22: LEFT_THUMB
Landmark 23: RIGHT_THUMB
Landmark 24: LEFT_HIP
Landmark 25: RIGHT_HIP
Landmark 26: LEFT_KNEE
Landmark 27: RIGHT_KNEE
Landmark 28: LEFT_ANKLE
Landmark 29: RIGHT_ANKLE
Landmark 30: LEFT_HEEL
Landmark 31: RIGHT_HEEL
Landmark 32: LEFT_FOOT_INDEX
Landmark 33: RIGHT_FOOT_INDEX


In [5]:
# Array Landmark yang digunakan
landmark_dipakai = [
    mp_pose.PoseLandmark.LEFT_SHOULDER,
    mp_pose.PoseLandmark.LEFT_ELBOW,
    mp_pose.PoseLandmark.LEFT_WRIST,
    mp_pose.PoseLandmark.LEFT_HIP,
    mp_pose.PoseLandmark.LEFT_KNEE,
    mp_pose.PoseLandmark.LEFT_ANKLE,
    mp_pose.PoseLandmark.RIGHT_SHOULDER,
    mp_pose.PoseLandmark.RIGHT_ELBOW,
    mp_pose.PoseLandmark.RIGHT_WRIST,
    mp_pose.PoseLandmark.RIGHT_HIP,
    mp_pose.PoseLandmark.RIGHT_KNEE,
    mp_pose.PoseLandmark.RIGHT_ANKLE
]

# Looping cetak nilai setiap Landmark yang digunakn
for idx, lnmark in enumerate(landmark_dipakai):
    print(f"Landmark {idx + 1}: {lnmark.name}\n{landmarks[lnmark.value]}")

Landmark 1: LEFT_SHOULDER
x: 0.6382650136947632
y: 0.8176428079605103
z: -0.14144879579544067
visibility: 0.9997240900993347

Landmark 2: LEFT_ELBOW
x: 0.8252437710762024
y: 0.7553074359893799
z: -0.34918689727783203
visibility: 0.9976186156272888

Landmark 3: LEFT_WRIST
x: 0.7308114767074585
y: 0.46343520283699036
z: -0.5030876994132996
visibility: 0.9936028122901917

Landmark 4: LEFT_HIP
x: 0.6058539152145386
y: 1.4332122802734375
z: -0.021652547642588615
visibility: 0.048822369426488876

Landmark 5: LEFT_KNEE
x: 0.5961827635765076
y: 1.9288995265960693
z: 0.1343768835067749
visibility: 0.002830507466569543

Landmark 6: LEFT_ANKLE
x: 0.5918279886245728
y: 2.341193914413452
z: 0.5873003005981445
visibility: 0.00010098393977386877

Landmark 7: RIGHT_SHOULDER
x: 0.3917210102081299
y: 0.8067614436149597
z: -0.1622365415096283
visibility: 0.9997650980949402

Landmark 8: RIGHT_ELBOW
x: 0.1947346329689026
y: 0.6825571060180664
z: -0.2881656885147095
visibility: 0.999755322933197

Landmark 9

Bandingkan sudut gambar target dengan sudut yang ada pada live video feed

In [6]:
def perbandingan(image, angles_target, angles_user):
    # tidak lebih dari 10 derajat toleransi (10-15 rekomndasi sudut toleransi)
    toleransi = 10
    angles_target = np.array(angles_target)
    angles_user = np.array(angles_user)
    cv2.putText(image, 'Keterangan', (220,14), cv2.FONT_HERSHEY_SIMPLEX, 0.7, [0,0,255], 1, cv2.LINE_AA)
    
    # Inisialisasi variabel hasil
    hasil = []
    
    # Membandinkan sudut pada target dan user dengan toleransi 10 derajat (10 + 10)
    # Siku Kiri
    if angles_user[0] < (angles_target[0] - toleransi):
        hasil.append(f"Rentangkan siku kiri anda")
        cv2.putText(image, str("Rentangkan siku kiri anda"), (220,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[0] > (angles_target[0] + toleransi):
        hasil.append(f"Tekuk siku kiri anda")
        cv2.putText(image, str("Tekuk siku kiri anda"), (220,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[0] >= (angles_target[0] - toleransi) and angles_user[0] <= (angles_target[0] + toleransi):
        hasil.append(f"Pass")
        cv2.putText(image, str("Pass"), (220,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,153,0], 1, cv2.LINE_AA)

    # Siku Kanan
    if angles_user[1] < (angles_target[1] - toleransi):
        hasil.append(f"Rentangkan siku Kanan anda")
        cv2.putText(image, str("Rentangkan siku Kanan anda"), (220,70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[1] > (angles_target[1] + toleransi):
        hasil.append(f"Tekuk siku Kanan anda")
        cv2.putText(image, str("Tekuk siku Kanan anda"), (220,70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[1] >= (angles_target[1] - toleransi) and angles_user[1] <= (angles_target[1] + toleransi):
        hasil.append(f"Pass")
        cv2.putText(image, str("Pass"), (220,70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,153,0], 1, cv2.LINE_AA)

    # Bahu Kiri
    if angles_user[2] < (angles_target[2] - toleransi):
        hasil.append(f"Angkat tangan kiri anda")
        cv2.putText(image, str("Angkat tangan kiri anda"), (220,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[2] > (angles_target[2] + toleransi):
        hasil.append(f"Turunkan tangan kiri anda")
        cv2.putText(image, str("Turunkan tangan kiri anda"), (220,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[2] >= (angles_target[2] - toleransi) and angles_user[2] <= (angles_target[2] + toleransi):
        hasil.append(f"Pass")
        cv2.putText(image, str("Pass"), (220,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,153,0], 1, cv2.LINE_AA)

    # Bahu Kanan
    if angles_user[3] < (angles_target[3] - toleransi):
        hasil.append(f"Angkat tangan Kanan anda")
        cv2.putText(image, str("Angkat tangan Kanan anda"), (220,130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[3] > (angles_target[3] + toleransi):
        hasil.append(f"Turunkan tangan Kanan anda")
        cv2.putText(image, str("Turunkan tangan Kanan anda"), (220,130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[3] >= (angles_target[3] - toleransi) and angles_user[3] <= (angles_target[3] + toleransi):
        hasil.append(f"Pass")
        cv2.putText(image, str("Pass"), (220,130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,153,0], 1, cv2.LINE_AA)

    # Pinggul Kiri
    if angles_user[4] < (angles_target[4] - toleransi):
        hasil.append(f"Angkat kaki kiri anda")
        cv2.putText(image, str("Angkat kaki kiri anda"), (220,160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[4] > (angles_target[4] + toleransi):
        hasil.append(f"Turunkan kaki kiri anda")
        cv2.putText(image, str("Turunkan kaki kiri anda"), (220,160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[4] >= (angles_target[4] - toleransi) and angles_user[4] <= (angles_target[4] + toleransi):
        hasil.append(f"Pass")
        cv2.putText(image, str("Pass"), (220,160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,153,0], 1, cv2.LINE_AA)

    # Pinggul Kanan
    if angles_user[5] < (angles_target[5] - toleransi):
        hasil.append(f"Angkat kaki Kanan anda")
        cv2.putText(image, str("Angkat kaki Kanan anda"), (220,190), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[5] > (angles_target[5] + toleransi):
        hasil.append(f"Turunkan kaki Kanan anda")
        cv2.putText(image, str("Turunkan kaki Kanan anda"), (220,190), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[5] >= (angles_target[5] - toleransi) and angles_user[5] <= (angles_target[5] + toleransi):
        hasil.append(f"Pass")
        cv2.putText(image, str("Pass"), (220,190), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,153,0], 1, cv2.LINE_AA)

    # Lutut Kiri
    if angles_user[6] < (angles_target[6] - toleransi):
        hasil.append(f"Luruskan lutut kiri anda")
        cv2.putText(image, str("Luruskan lutut kiri anda"), (220,220), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[6] > (angles_target[6] + toleransi):
        hasil.append(f"Tekuk lutut kiri anda")
        cv2.putText(image, str("Tekuk lutut kiri anda"), (220,220), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[6] >= (angles_target[6] - toleransi) and angles_user[6] <= (angles_target[6] + toleransi):
        hasil.append(f"Pass")
        cv2.putText(image, str("Pass"), (220,220), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,153,0], 1, cv2.LINE_AA)

    # Lutut Kanan
    if angles_user[7] < (angles_target[7] - toleransi):
        hasil.append(f"Luruskan Lutut Kanan anda")
        cv2.putText(image, str("Luruskan Lutut Kanan anda"), (220,250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[7] > (angles_target[7] + toleransi):
        hasil.append(f"Tekuk Lutut Kanan anda")
        cv2.putText(image, str("Tekuk Lutut Kanan anda"), (220,250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [3,119,252], 1, cv2.LINE_AA)
    elif angles_user[7] >= (angles_target[7] - toleransi) and angles_user[7] <= (angles_target[7] + toleransi):
        hasil.append(f"Pass")
        cv2.putText(image, str("Pass"), (220,250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,153,0], 1, cv2.LINE_AA)

    return hasil
    

Program Real-time Video Feed

In [7]:
# Objektif testing
# 1. Mengambil sebuah frame gambar dari rekaman real-time dengan kodisi tertentu.
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Atur resolusi camera (Default 640x480)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1024)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

frame_dieksekusi = False

# Cetak resolusi camera
if cap.isOpened():
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print('Ukuran kamera: {} x {}'.format(width, height))

    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Simpan sudut - sudut kedalam array
            angles = []
            hasil = []

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get landmarks coordinates
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                # Hitung sudut (vector)
                angle1 = calculate_angle(left_shoulder, left_elbow, left_wrist) # Siku kiri
                angles.append(angle1)

                angle2 = calculate_angle(right_shoulder, right_elbow, right_wrist) # Siku kanan
                angles.append(angle2)

                angle3 = calculate_angle(left_elbow, left_shoulder, left_hip) # Bahu kiri
                angles.append(angle3)

                angle4 = calculate_angle(right_elbow, right_shoulder, right_hip) # Bahu kanan
                angles.append(angle4)

                angle5 = calculate_angle(left_shoulder, left_hip, left_knee) # Pinggul kiri
                angles.append(angle5)

                angle6 = calculate_angle(right_shoulder, right_hip, right_knee) # Pinggul kanan
                angles.append(angle6)

                angle7 = calculate_angle(left_hip, left_knee, left_ankle) # Lutut kiri
                angles.append(angle7)

                angle8 = calculate_angle(right_hip, right_knee, right_ankle) # Lutut kanan
                angles.append(angle8)

                # Visualisasi Legenda
                # cv2.rectangle(image,(0,0), (480,260), (255,255,255), -1)

                cv2.putText(image, 'Nama', (10,14), cv2.FONT_HERSHEY_SIMPLEX, 0.7, [0,0,255], 1, cv2.LINE_AA)
                cv2.putText(image, str('Siku Siki'), (10,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str('Siku Kanan'), (10,70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str('Bahu Kiri'), (10,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str('Bahu Kanan'), (10,130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str('Pinggul Kiri'), (10,160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str('Pinggul Kanan'), (10,190), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str('Lutut Kiri'), (10,220), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str('Lutut Kanan'), (10,250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)

                cv2.putText(image, 'Sudut', (140,14), cv2.FONT_HERSHEY_SIMPLEX, 0.7, [0,0,255], 1, cv2.LINE_AA)
                cv2.putText(image, str(int(angle1)), (140,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str(int(angle2)), (140,70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str(int(angle3)), (140,100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str(int(angle4)), (140,130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str(int(angle5)), (140,160), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str(int(angle6)), (140,190), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str(int(angle7)), (140,220), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)
                cv2.putText(image, str(int(angle8)), (140,250), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255, 255, 255], 1, cv2.LINE_AA)

                hasil = perbandingan(image, angles_target, angles)

                # # Visualisasi sudut siku kiri
                # cv2.putText(image, str(angle1), 
                #                tuple(np.multiply(left_elbow, [1280, 720]).astype(int)), 
                #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                        
                # # Visualisasi sudut siku kanan
                # cv2.putText(image, str(angle2), 
                #                tuple(np.multiply(right_elbow, [1280, 720]).astype(int)), 
                #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # # Visualisasi sudut bahu kiri
                # cv2.putText(image, str(angle1), 
                #                tuple(np.multiply(left_shoulder, [1280, 720]).astype(int)), 
                #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                        
                # # Visualisasi sudut bahu kanan
                # cv2.putText(image, str(angle2), 
                #                tuple(np.multiply(right_shoulder, [1280, 720]).astype(int)), 
                #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                # # Visualisasi sudut pinggul kiri
                # cv2.putText(image, str(angle3), 
                #                tuple(np.multiply(left_hip, [1280, 720]).astype(int)), 
                #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # # Visualisasi sudut pinggul kanan
                # cv2.putText(image, str(angle4), 
                #                tuple(np.multiply(right_hip, [1280, 720]).astype(int)), 
                #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # # Visualisasi sudut lutut kiri
                # cv2.putText(image, str(angle5), 
                #                tuple(np.multiply(left_knee, [1280, 720]).astype(int)), 
                #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # # Visualisasi sudut lutut kanan
                # cv2.putText(image, str(angle6), 
                #                tuple(np.multiply(right_knee, [1280, 720]).astype(int)), 
                #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                # print("Hasil dari perbandingan:", hasil)
                    
            except:
                pass

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )  
                         
            if all(item == "Pass" for item in hasil) and not frame_dieksekusi :
                    print(f"Berhasil memenuhi kriteria")
                    print("Sudut pada gambar real-time", angles)
                    angles_rt = angles.copy()

                    # Simpan frame ke dalam folder dengan format .jpg
                    current_datetime = datetime.now()  # Ambil tanggal dan waktu sekarang
                    timestamp = current_datetime.strftime('%Y-%m-%d_%H-%M-%S')  # Atur format tanggal dan ubah ke string

                    output_image_path = f'Asset/Setelah Diproses/Gambar Real-Time/Gambar_Processed_{timestamp}.jpg' #Ubah direktori untuk menyimpan gambar
                    cv2.imwrite(output_image_path, image)
                    print(f"Processed image saved as '{output_image_path}'")
                    frame_dieksekusi = True
                    
            cv2.imshow('Mediapipe Video Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            
    cap.release()
    cv2.destroyAllWindows()
else:
    print('Tidak dapat membuka kamera')    

Ukuran kamera: 1280 x 720
Berhasil memenuhi kriteria
Sudut pada gambar real-time [96, 109, 106, 117, 178, 178, 179, 178]
Processed image saved as 'Asset/Setelah Diproses/Gambar Real-Time/Gambar_Processed_2024-02-15_20-50-35.jpg'


Metode Cosine Similarity

In [8]:
# Cosine Similarity
# Definisikan dua vektor
vector1 = angles_target
vector2 = angles_rt

# Konversi vektor ke dalam numpy array
vector1 = np.array(vector1)
vector2 = np.array(vector2)

# Hitung dot product
dot_product = np.dot(vector1, vector2)

# Hitung magnitude (panjang) dari masing-masing vektor
magnitude_vector1 = np.linalg.norm(vector1)
magnitude_vector2 = np.linalg.norm(vector2)

# Hitung cosine similarity
cosine_similarity = dot_product / (magnitude_vector1 * magnitude_vector2) * 100

print("Sudut pada gambar target", angles_target)
print("Sudut pada gambar real-time", angles_rt)
print("Cosine Similarity :", round(cosine_similarity, 2))

Sudut pada gambar target [91, 99, 111, 126, 178, 176, 179, 180]
Sudut pada gambar real-time [96, 109, 106, 117, 178, 178, 179, 178]
Cosine Similarity : 99.93


Metode Jaccard Similarity

In [9]:
# # Definisikan dua set (atau array)
# set1 = angles_target
# set2 = angles_rt

# # Hitung intersection (elemen yang sama dalam kedua set)
# intersection = len(set1.intersection(set2))

# # Hitung union (elemen unik dalam kedua set)
# union = len(set1.union(set2))

# # Hitung Jaccard Similarity
# jaccard_similarity = intersection / union

# print("Jaccard Similarity:", jaccard_similarity)

Metode Euclidean Distance

In [10]:
vector1 = angles_target
vector2 = angles_rt

# Konversi vektor ke dalam numpy array
vector1 = np.array(vector1)
vector2 = np.array(vector2)

euclidean_distance = np.linalg.norm(vector1 - vector2)
print("Euclidean Distance :", round(euclidean_distance, 2))

Euclidean Distance : 15.46


Metode Manhattan Distance

In [11]:
vector1 = angles_target
vector2 = angles_rt

# Konversi vektor ke dalam numpy array
vector1 = np.array(vector1)
vector2 = np.array(vector2)

# Calculate Manhattan distance
manhattan_distance = sum(abs(a - b) for a, b in zip(vector1, vector2))

print("Manhattan Distance :", manhattan_distance)

Manhattan Distance : 33


Metode Minkowski Distance

In [42]:
# import numpy as np

# def minkowski_distance(x, y, p):
#     return np.power(np.sum(np.power(np.abs(x - y), p)), 1/p)

# vector1 = np.array(angles_target)
# vector2 = np.array(angles_rt)

# # Hitung Manhattan distance (p=1)
# manhattan_dist = minkowski_distance(vector1, vector2, 1)
# print("Manhattan Distance:", round(manhattan_dist, 2))

# # Hitung Euclidean distance (p=2)
# euclidean_dist = minkowski_distance(vector1, vector2, 2)
# print("Euclidean Distance:", round(euclidean_dist, 2))

Mengubah nilai Eucledian dan Manhattan Distance menjadi persenan akurasi.

In [12]:
# MaxED dan MaxMD
# Definisikan vektor1 (nilai maksimum) dan vector2 (nilai minimum)
vector1 = np.array([180, 180, 180, 180, 180, 180, 180, 180]) # karena hasil maksimal setiap dimensi adalah 180 derajat (setengah lingkaran)
vector2 = np.array([0, 0, 0, 0, 0, 0, 0, 0])

# MaxED
max_euclidean_distance = np.linalg.norm(vector1 - vector2)

# MaxMD
max_manhattan_distance = sum(abs(a - b) for a, b in zip(vector1, vector2))

print("Maximum Euclidean Distance :", round(max_euclidean_distance, 2))
print("Maximum Manhattan Distance :", round(max_manhattan_distance, 2))

Maximum Euclidean Distance : 509.12
Maximum Manhattan Distance : 1440


In [13]:
# Nilai Euclidean Distance dan Manhattan Distance yang telah dihitung
Euclidean_Distance = euclidean_distance
print(f"Euclidean Distance :", round(Euclidean_Distance, 2))

Manhattan_Distance = manhattan_distance
print(f"Manhattan Distance :", round(Manhattan_Distance, 2))

# Nilai minimum dan maksimum yang mungkin
Minimum_Value = 0

Maximum_Value_Eucledian = max_euclidean_distance
print(f"Maximum Euclidean Distance :", round(Maximum_Value_Eucledian, 2))

Maximum_Value_Manhattan = max_manhattan_distance
print(f"Maximum Manhattan Distance :", round(Maximum_Value_Manhattan, 2))

# Normalisasi Euclidean Distance
Euclidean_Distance_Percentage = (Maximum_Value_Eucledian - Euclidean_Distance) / (Maximum_Value_Eucledian - Minimum_Value) * 100

# Normalisasi Manhattan Distance
Manhattan_Distance_Percentage = (Maximum_Value_Manhattan - Manhattan_Distance) / (Maximum_Value_Manhattan - Minimum_Value) * 100

# Menampilkan hasil Persentasi
print(f"Euclidean Distance Percentage :", round(Euclidean_Distance_Percentage, 2),"%")

print(f"Manhattan Distance Percentage :", round(Manhattan_Distance_Percentage, 2),"%")

Euclidean Distance : 15.46
Manhattan Distance : 33
Maximum Euclidean Distance : 509.12
Maximum Manhattan Distance : 1440
Euclidean Distance Percentage : 96.96 %
Manhattan Distance Percentage : 97.71 %
